In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Training

In [ ]:
metrics = pd.read_csv("metrics_stft.csv")

# Add index column
metrics["index"] = metrics.index
metrics

In [ ]:
# Plotting the data, with the x-axis being the number of epochs and the y-axis being the scores of reward, audio_sim and transcription_sim
# fig, ax = plt.subplots()
# ax.set(xlabel="Epochs", ylabel="Scores")
plt.title("Scores over epochs", fontsize=20)

sns.set_theme(style="whitegrid", palette="deep")

plt.xlabel("Epochs")
plt.locator_params(axis='x', integer=True)

data = metrics.melt('index', var_name='score', value_name='value')
sns.lineplot(x='index', y='value', hue='score', data=data)

In [ ]:
DELAY = 100

fig, axes = plt.subplots(3, 1, figsize=(15, 8), sharex=True)
fig.suptitle('MODEL DDPG Training Results')

sns.lineplot(ax=axes[0], x="index", y="reward", data=metrics, color="r", alpha= 0.4)
reward_mean = [np.mean(metrics["reward"][i-DELAY:i]) for i in range(DELAY, len(metrics))]
sns.lineplot(ax=axes[0], x=range(DELAY, len(metrics)), y=reward_mean, color="black")
axes[0].set_ylabel("Reward")

sns.lineplot(ax=axes[1], x="index", y="transcription_sim", data=metrics, color="g", alpha= 0.4)
transcription_mean = [np.mean(metrics["transcription_sim"][i-DELAY:i]) for i in range(DELAY, len(metrics))]
sns.lineplot(ax=axes[1], x=range(DELAY, len(metrics)), y=transcription_mean, color="black")
axes[1].set_ylabel("Transcription Similarity")

sns.lineplot(ax=axes[2], x="index", y="audio_dissim", data=metrics, color="b", alpha= 0.4)
audio_mean = [np.mean(metrics["audio_dissim"][i-DELAY:i]) for i in range(DELAY, len(metrics))]
sns.lineplot(ax=axes[2], x=range(DELAY, len(metrics)), y=audio_mean, color="black")
axes[2].set_ylabel("Audio Dissimilarity")

axes[2].set_xlabel("Iteration")

# Align y-axis labels
fig.align_ylabels()
plt.show()

A latex table showing mean, std for each of the areas.

In [ ]:
reward_table_mean = metrics["reward"].mean()
reward_table_std  = metrics["reward"].std()

trans_table_mean = metrics["transcription_sim"].mean()
trans_table_std  = metrics["transcription_sim"].std()

dissim_table_mean = metrics["audio_dissim"].mean()
dissim_table_std  = metrics["audio_dissim"].std()

POI = 2

table_start =   "\\begin{table}[h!]\n" + \
                "   \\begin{center}\n" + \
                "   \\begin{tabular}{ c || c |  c }\n" + \
                "       Metric & mean & Standard Deviation \\\\\\hline\n"

table_content = f"       Reward & {round(reward_table_mean, POI)} & {round(reward_table_std, POI)}\\\\\n" + \
                f"       Transcription Similarity & {round(trans_table_mean, POI)} & {round(trans_table_std, POI)}\\\\\n" + \
                f"       Audio Dissimilarity & {round(dissim_table_mean, POI)} & {round(dissim_table_std, POI)}\\\\\n"

table_end =     "   \\end{tabular}\n" + \
                "   \\caption{Caption}\n" + \
                "   \\label{table:Table}\n" + \
                "   \\end{center}\n" + \
                "\\end{table}\n"
print(table_start + table_content + table_end)

## Attacks

In [ ]:
data = pd.read_csv('metrics_compare.csv')
# Remove audio_path
data.drop('audio_path', axis=1, inplace=True)
data.groupby('attack_type').mean()

#Normalize the data (audio_distance and similarity)
data['Audio Dissimilarity'] = data['audio_distance'] / data['audio_distance'].max()
data['Transcription Similarity'] = data['similarity']

data.drop("similarity", axis=1,inplace=True) 
data.drop("audio_distance", axis=1,inplace=True) 

# Rename sucess to Sucess
data["Success"] = data["success"]
data.drop("success", axis=1,inplace=True) 

data = data.melt(id_vars=['attack_type'], value_vars=['Transcription Similarity', 'Audio Dissimilarity', 'Success'])


In [ ]:

sns.catplot(x='attack_type', 
                y='value', 
                hue='variable', 
                data=data, 
                kind='bar', 
                height=6, 
                aspect=1.5,
                palette="Set2")

# Rotate the x-axis labels
plt.xticks(rotation=45)

## Defence

In [ ]:
classify_db = pd.read_csv("defence/classify_detect_results.csv")


In [ ]:
print(classify_db.to_latex(index=False, float_format="%.2f"))

In [ ]:
compare_db = pd.read_csv("defence/compare_detect_results.csv")
compare_db

In [ ]:
from sklearn import metrics 

table_start =   "\\begin{table}[h!]\n" + \
                "   \\begin{center}\n" + \
                "   \\begin{tabular}{ c || c c c c}\n"
table_end =     "   \\end{tabular}\n" + \
                "   \\caption{Caption}\n" + \
                "   \\label{table:Table}\n" + \
                "   \\end{center}\n" + \
                "\\end{table}\n"

cmap = sns.color_palette("light:b", as_cmap=True)
for index in compare_db.index:
     fig, axes = plt.subplots()
     row = compare_db.loc[index]
     name = row["Attack"]
     name = name.translate({ord('_'): ' [', ord('/'): ']'})
     label = f"{name.capitalize()} Using Whisper and Google Speech Recognition as Defence"
     fig.supxlabel("Predicted label")
     fig.supylabel("True Label")
     axes.set_title(label)
     sns.heatmap([[row["FN"], row["FP"]], [row["TN"],row["TP"]] ], annot=True, fmt=".0f", ax=axes, cmap=cmap)
     plt.show()

print(table_start, end="")
print("\tAttack & Accuracy & Precision & Recall & F1\\\\\\hline\n", end='')
for index in compare_db.index:
     row = compare_db.loc[index]
     name = row["Attack"]
     name = name.translate({ord('_'): ' ', ord('/'): ''})
     name = name.split(sep=' ')[1]
     print(f"\t{name} & {row['Accuracy']} & {row['Precision']} & {row['Recall']} & {row['F1']} \\\\\n", end="")
print(table_end)
    